In [5]:
import numpy as np
import pandas as pd
import time
from glob import glob

from data_provider import read_data_path, read_prices, read_data
from backtest import simulation
np.set_printoptions(edgeitems=30, linewidth=1000, formatter=dict(float=lambda x: "%.7g" % x))

In [12]:
#timeframes = ['1m', '3m', '5m', '15m', '24m', '30m', '1h']
timeframes = ['3m', '5m', '15m', '24m', '30m', '1h']
timeframes = ['1h']
pair = 'USDT'
data_path = read_data_path()
data_path = data_path + '/' + pair 
files = glob(data_path + '/*')
markets = [x.split('/')[-1] for x in files]
#markets = ['BTC-USDT']
#markets = ['EOS-BTC','TRX-BTC', 'ONT-BTC', 'ZIL-BTC', 'VEN-BTC']

In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from statsmodels.tsa.arima_model import ARIMA

for c in d_f.columns:
        d_f[c+'_ret'] = d_f[c].pct_change(2).fillna(0)
           
sc = StandardScaler()
#d_f['Gold_ret'] = sc.fit_transform(d_f['Gold_ret'])
#402
#113
#302
step = 30
for ai in range(0, len(d_f), step):
    df = d_f.loc[d_f.index[ai:ai+step], :]
    x = df['Gold_ret']
    x_min = min(x)
    x_max = max(x)
#    x = (x - x_min)/(x_max-x_min)
#    sc = StandardScaler()
#    x = sc.fit_transform(x)
    ##based on model AIC
    min_aic = np.inf
    best_params = (0,0,0)
    for i in range(5):
        for j in range(5):
            for k in range(5):
                try:
                    arima = ARIMA(x, order = (i,j,k)).fit()
#                    print (i,j,k, arima.aic)
                    if arima.aic < min_aic:
                        min_aic = arima.aic
                        best_params = (i,j,k)
                except:
                    pass
    #            
    arima = ARIMA(x, order = best_params).fit()
    print ('AIC of ARIMA model', arima.aic)
    print ('Params of ARIMA model', best_params)
    
    x_pred = arima.fittedvalues
#    x_pred = x_pred*(x_max - x_min) + x_min
    d_f.loc[df.index, 'Pred_ret'] = x_pred#arima.fittedvalues
df = d_f
df['Pred'] = df.Gold
for j in range(len(df.index)):
    if j < 2: 
        continue
    i = df.index[j]
    prev = df.index[j-2]
    df.loc[i, 'Pred'] = df.loc[prev, 'Pred']*(1 + df.loc[i, 'Pred_ret'])
df.to_hdf('DeepLearning.h5', 'Pred_ARIMA')
plt.plot(df['Gold'])
plt.plot(df['Pred'], label = 'pred')
plt.legend()
plt.show()

In [ ]:
init_data = {'exchange_commission': 0.0005,
             'start_capital'     : 100,
             'buy_correction'    : 1.0001,
             'sell_correction'   : 0.9999,
             'robot_name'        : '',
             'market'            : '',
             'kline'             : ''  
            }

lag = 24
start_time = time.time()
res = []
for market in markets:
    for kline in timeframes:
        prices = read_prices(market, kline)
        returns = np.diff(np.log(prices))
        data = []
        for k in range(lag):
            xx = np.roll(returns, k)
            data.append(xx)

        data = np.array(data)
        data = data.transpose()
        data = data[lag-1:]
        data_binary = np.sign(data)
        labels = data_binary[:, 0]
        labels[labels == 0] = -1

        train = int(0.8 * data.shape[0])
        val = int(0 * (data.shape[0] - train)) + train

        X_train = data[:train, 1:]  
        y_train = labels[:train]

        X_test  = data[val:, 1:]
        y_test  = labels[val:]
        
        t1 = time.time()
        model = autosklearn_class(X_train, y_train)        
        t2 = time.time()
        
        y_pred = model.predict(X_test)
        acc = np.mean(y_pred == y_test) * 100
        print(market, kline,"accuracy:", round(acc,3), "time:", round((t2-t1)/60), 'min')
        
        signals = y_pred
        bt_prices = prices[-signals.shape[0]-1:-1]
        df = simulation(bt_prices, signals, init_data)
        
        res.append([market, kline,  acc, df.iloc[-1]['cum_profit']])
        df = pd.DataFrame(res)
        df.to_csv('Logs/res_autosklearn-class_' + market + '_' + kline + '.csv')
print("Time taken = {0:.5f}".format(time.time() - start_time))

In [ ]:

start_time = time.time()
res = []
for market in markets:
    for kline in timeframes:
        prices = read_prices(market, kline)
        returns = np.diff(np.log(prices))
        returns = returns[returns!=0]
        data = []
        for k in range(lag):
            xx = np.roll(returns, k)
            data.append(xx)

        data = np.array(data)
        data = data.transpose()
        data = data[lag-1:]
        data_binary = np.sign(data)

        train = int(0.7 * data.shape[0])

        y_train = data_binary[:train, 0]
        X_train = data[:train, 1:]
        y_val   = data_binary[train:, 0]
        X_val   = data[train:, 1:]


        best_model, best_score, all_models, all_scores = bestFit(RandomForestClassifier(), paramGrid,
                                                    X_train, y_train, X_val, y_val, nfolds=5, #[optional, instead of validation set]
                                                    metric=roc_auc_score, greater_is_better=True, 
                                                    scoreLabel='AUC',
                                                    showPlot = False)
        print(market, kline, round(best_score,3))
        res.append([market, kline, best_score])
        df = pd.DataFrame(res)
        df.to_csv('Logs/res_pr-sklearn_' + market + '_' + kline + '.csv')
print("Time taken = {0:.5f}".format(time.time() - start_time))

In [16]:
xx = np.arange(20).reshape(-1,4)
print(xx)
np.roll(xx,1, axis = 0)


[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]]


array([[16, 17, 18, 19],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [18]:
market = 'BTC-USDT'
kline = '1h'
lag = 30
window = 30
window_step = 1
forecast = 1

df= read_data(market, kline)

data = df.as_matrix(['O','H', 'L', 'C', 'V'])
data[:,4] = data[:,4] + 1 
data  = np.diff(np.log(data), axis = 0)

X = []
Y = []
for i in range(0, len(data) , window_step): 
    try:
        data_win = data[i:i + window, :].reshape(-1)
        X.append(data_win)
        Y.append(data[i + window + forecast, 3])
    except:
        break
X = X[:len(Y)]
X = np.array(X)
Y = np.array(Y)
Y = np.sign(Y)

train = int(0.6 * X.shape[0])
val = int(0.5 * (X.shape[0] - train)) + train

y_train = Y[:train]
X_train = X[:train,:]
y_val  = Y[train:val]
X_val  = X[train:val,:]
y_test  = Y[val:]
X_test  = X[val:,:]



In [21]:
market = 'BTC-USDT'
kline = '1h'
lag = 30

prices= read_prices(market, kline)

returns = np.diff(np.log(prices))
returns = returns[returns!=0]
data = []
for k in range(lag):
    xx = np.roll(returns, k)
    data.append(xx)

data = np.array(data)
data = data.transpose()
data = data[lag-1:]
train = int(0.8 * data.shape[0])
val = int(0 * (data.shape[0] - train)) + train

train_rt = data[:train, 0]
mu = np.mean(train_rt)
v  = np.std(train_rt) 
bins = [mu-v, mu, mu+v]
data_bins   = np.digitize(data[:,1:], bins)

data_binary = np.sign(data)

y_train = data_binary[:train, 0]
X_train = data[:train, 1:]
y_val  = data_binary[train:val, 0]
X_val  = data[train:val, 1:]

y_test  = data_binary[val:, 0]
X_test  = data[val:, 1:]


#y_train = data_binary[:, 0]
#X_train = data[:, 1:]

print(X_train.shape[0] + X_val.shape[0] + X_test.shape[0])


2120


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid

from parfit import bestFit # Necessary if you wish to use bestFit
from sklearn.metrics import accuracy_score
from parfit.score import roc_auc_score

grid = {
    'min_samples_leaf': [1, 2, 5, 20, 50, 100],
    'max_features': ['log2', 0.5, 0.55, 0.6, 0.7],
    'n_estimators': [40, 60, 100, 500],
    'n_jobs':[-1],
}
paramGrid = ParameterGrid(grid)

ts = time.time()
best_model, best_score, all_models, all_scores = bestFit(RandomForestClassifier(), paramGrid,
                                                    #X_train, y_train, X_val, y_val,  #[optional, instead of cross - validation]
                                                    X_train, y_train, nfolds=5, #[optional, instead of validation set]
                                                    greater_is_better=True,
                                                    #metric=accuracy_score,                                                          
                                                    #scoreLabel='Accuracy', 
                                                    metric=roc_auc_score,      
                                                    scoreLabel='AUC',
                                                    showPlot = False)

tt = round(time.time() - ts)
print(best_score, tt)

In [ ]:
print(best_model)
pred = best_model.predict(X_test)
np.mean(pred == y_test)

3596


In [18]:
pred = automl.predict(X_test)
np.mean(pred == y_test)

0.5731132075471698

In [19]:
automl

AutoSklearnClassifier(delete_output_folder_after_terminate=True,
           delete_tmp_folder_after_terminate=True,
           disable_evaluator_output=False, ensemble_nbest=50,
           ensemble_size=50, exclude_estimators=None,
           exclude_preprocessors=None, get_smac_object_callback=None,
           include_estimators=None, include_preprocessors=None,
           initial_configurations_via_metalearning=25,
           ml_memory_limit=3072, output_folder=None,
           per_run_time_limit=360, resampling_strategy='holdout',
           resampling_strategy_arguments=None, seed=1, shared_mode=False,
           smac_scenario_args=None, time_left_for_this_task=3600,
           tmp_folder=None)

In [6]:
bt_prices = prices[train:]
bt_returns = np.diff(np.log(bt_prices))

data = []
for k in range(lag):
    xx = np.roll(bt_returns, k)
    data.append(xx)
    
data = np.array(data)
data = data.transpose()
data = data[lag-1:]

ticks = bt_prices[lag-1:]
np.diff(np.log(ticks))
pred = best_model.predict(data[:,1:])
true_moves = np.sign(data[:,0])
#pred = true_moves
signals = np.zeros(len(pred) +1)
signals[:-1] = pred

In [8]:
from backtest import simulation
init_data = {'exchange_commission': 0.0005,
             'start_capital'     : 100,
             'buy_correction'    : 1.0001,
             'sell_correction'   : 0.9999,
             'robot_name'        : '',
             'market'            : '',
             'kline'             : ''  
            }

df = simulation(ticks, signals, init_data)
print('days', round(len(ticks)/24))
print('trades', df[df['signal'] == -1].shape[0])
df

days 34
trades 201


,price,signal,MP,capital,num_shares,profit,cum_profit
0,8779.97,0,0,100.000000000,0.000000000,0.000000000,0.000000000
1,8792.47,0,0,100.000000000,0.000000000,0.000000000,0.000000000
2,8765.00,1,1,0.000000000,0.011402168,-100.000000000,0.000000000
3,8780.01,0,1,0.000000000,0.011402168,-100.000000000,0.000000000
4,8806.29,-1,0,100.350560072,0.000000000,0.350560072,0.350560072
5,8915.31,1,1,0.000000000,0.011249228,-100.350560072,0.350560072
6,8816.54,0,1,0.000000000,0.011249228,-100.350560072,0.350560072
7,8815.99,0,1,0.000000000,0.011249228,-100.350560072,0.350560072
8,8852.05,-1,0,99.518987328,0.000000000,-0.831572744,-0.481012672
9,8811.00,1,1,0.000000000,0.011288081,-99.518987328,-0.481012672


In [ ]:
def simulation_short(ticks, signals, init_data):
    market          = init_data['market']
    commission      = init_data['exchange_commission']
    start_capital   = init_data['start_capital']
    buy_correction  = init_data['buy_correction']
    sell_correction = init_data['sell_correction']
     
    cur_capital  = start_capital 
    num_shares   = 0
    trade_profit = 0
    cum_profit   = 0
    
    market_position = 0
    start = len(ticks) - len(signals) 
    ticks = ticks[start:]
    trade_statistics = []
    #trade_statistics = np.zeros((len(signals),7))
    for i in range(len(signals)) :
        
        if market_position = 0  and signals[i] == 1:
            price = ticks[i] * buy_correction
            num_shares   = cur_capital * (1-commission)/price
            trade_profit = -cur_capital
            cur_capital  = 0
            market_position = market_position + 1
              
        elif market_position = 1 and signals[i] == -1:
            price = ticks[i] * sell_correction
            cur_capital  = (num_shares * price)*(1-commission)
            trade_profit = trade_profit + cur_capital
            cum_profit  += trade_profit
            num_shares   = 0
            market_position = market_position - 1
            
        elif market_position = 0 and signals[i] == -1:
            price = ticks[i] * sell_correction
            num_shares   = cur_capital * (1-commission)/price
            trade_profit = cur_capital
            cum_profit  += trade_profit
            market_position = market_position - 1
        elif market_position = -1 and signals[i] == 1:
            price = ticks[i] * buy_correction
            cur_capital  = (num_shares * price)*(1-commission)
            trade_profit = -cur_capital
            cum_profit  += trade_profit
            market_position = market_position - 1                  
        else:
            signals[i] = 0
            
        trade_statistics.append([ticks[i], signals[i], market_position, cur_capital, num_shares, trade_profit, cum_profit])            
        #trade_statistics[i] = [ticks[i], signals[i], market_position, cur_capital, num_shares, trade_profit, cum_profit]

    df = pd.DataFrame(trade_statistics)
    df.columns = ['price', 'signal', 'MP', 'capital', 'num_shares', 'profit', 'cum_profit']
    df = df.astype({'signal':np.int32, 'MP':np.int32})
    return df

In [13]:
print('days', round(len(ticks)/24))
print('trades', df[df['signal'] == -1].shape[0])
df;

days 34
trades 201


In [ ]:
import xgboost as xgb

In [ ]:
clf = xgb.XGBClassifier(n_jobs = -1, max_depth = 3)
clf.fit(X_train, y_train)
pred = clf.predict(X_val)
print(np.mean(pred == y_val))
clf.get_xgb_params()

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y_train_oh = lb.fit_transform(y_train)
churn_dmatrix = xgb.DMatrix(data=X_train, label=y_train_oh)
params = {"objective":"binary:logistic", "max_depth":3, "learning_rate": 0.03, "n_estimators" :1000}
cv_results = xgb.cv(dtrain=churn_dmatrix, params= params, nfold=4, num_boost_round=1000, metrics="error", early_stopping_rounds=50)

print(cv_results)
print(((1-cv_results["test-error-mean"]).iloc[-1]))

In [ ]:
cv_results = xgb.cv(dtrain=churn_dmatrix, params= params, nfold=3, num_boost_round=5, metrics="auc", as_pandas=True, seed=123)
print(cv_results)
print((cv_results["test-auc-mean"]).iloc[-1])

In [ ]:
clf = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

xgb_param = clf.get_xgb_params()
xgtrain = xgb.DMatrix(data=X_train, label=y_train_oh)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=clf.get_params()['n_estimators'], nfold=5,
        metrics='auc', early_stopping_rounds=50)


In [ ]:
    clf.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')